(databases)=
# 데이터베이스

## 소개

엄청난 양의 데이터가 데이터베이스에 저장되어 있으므로 데이터베이스에 액세스하는 방법을 아는 것이 중요합니다. 때로는 누군가에게 .csv로 스냅샷을 다운로드하도록 요청할 수 있지만 이는 금방 고통스러워집니다. 변경해야 할 때마다 다른 사람과 소통해야 합니다. 필요할 때 필요한 데이터를 얻기 위해 데이터베이스에 직접 접근할 수 있기를 원할 것입니다.

이 장에서는 먼저 SQL 데이터베이스와 상호 작용할 수 있는 파이썬 패키지의 기본 사항, 즉 데이터베이스에 연결한 다음 SQL[^import-databases-1] 쿼리로 데이터를 검색하는 방법을 배웁니다. **SQL**(Structured Query Language의 약자)은 데이터베이스의 공용어이며 모든 데이터 과학자가 배워야 할 중요한 언어입니다. 이 장을 마치면 SQL 마스터가 되지는 않겠지만 가장 중요한 구성 요소를 식별하고 해당 구성 요소가 수행하는 작업을 이해할 수 있게 될 것입니다.

[^import-databases-1]: SQL은 "에스큐엘" 또는 "시퀄"로 발음됩니다.

### 전제 조건

이 장에서는 **pandas**, **SQLModel** 및 **ibis** 패키지가 필요합니다. 이미 **pandas**가 설치되어 있을 것입니다. **SQLModel** 및 **ibis**를 설치하려면 각각 컴퓨터 명령줄에서 `pip install sqlmodel` 및 `pip install ibis-framework`를 실행합니다. 먼저 몇 가지 일반 패키지를 가져오고 자세한 경고를 끕니다.

In [ ]:
import warnings
from pathlib import Path

warnings.filterwarnings("ignore")

## 데이터베이스 기본 사항

가장 간단한 수준에서 데이터베이스는 데이터베이스 용어로 **테이블**이라고 하는 데이터 프레임 모음으로 생각할 수 있습니다.
**pandas** 데이터 프레임과 마찬가지로 데이터베이스 테이블은 명명된 열 모음이며, 여기서 열의 모든 값은 동일한 유형입니다.
데이터 프레임과 데이터베이스 테이블 간에는 세 가지 높은 수준의 차이점이 있습니다.

-   데이터베이스 테이블은 디스크(즉, 파일)에 저장되며 임의로 클 수 있습니다.
    데이터 프레임은 메모리에 저장되며 근본적으로 제한됩니다(해당 제한은 여전히 많은 문제에 충분히 크지만). 디스크와 메모리의 차이는 장기 기억과 단기 기억의 차이와 같다고 생각할 수 있습니다(후자의 경우 용량이 훨씬 더 제한적임).

-   데이터베이스 테이블에는 거의 항상 인덱스가 있습니다.
    책의 색인과 마찬가지로 데이터베이스 인덱스를 사용하면 모든 단일 행을 살펴보지 않고도 관심 있는 행을 빠르게 찾을 수 있습니다.

-   대부분의 기존 데이터베이스는 기존 데이터를 분석하는 것이 아니라 데이터를 빠르게 수집하는 데 최적화되어 있습니다.
    이러한 데이터베이스는 데이터가 열별이 아닌 행별로 저장되기 때문에 **행 지향**이라고 합니다.
    최근에는 기존 데이터를 훨씬 빠르게 분석할 수 있는 **열 지향** 데이터베이스 개발이 많이 이루어졌습니다.

데이터베이스는 데이터베이스 관리 시스템(**DBMS** 약칭)에서 실행되며 세 가지 기본 형식이 있습니다.

-   *클라이언트-서버* DBMS는 강력한 중앙 서버에서 실행되며 컴퓨터(클라이언트)에서 연결합니다. 조직의 여러 사람과 데이터를 공유하는 데 유용합니다. 널리 사용되는 클라이언트-서버 DBMS에는 PostgreSQL, MariaDB, SQL Server 및 Oracle이 있습니다.
-   Snowflake, Amazon의 RedShift 및 Google의 BigQuery와 같은 *클라우드* DBMS는 클라이언트 서버 DBMS와 유사하지만 클라우드에서 실행됩니다. 즉, 매우 큰 데이터 세트를 쉽게 처리할 수 있고 필요에 따라 자동으로 더 많은 컴퓨팅 리소스를 제공할 수 있습니다.
-   SQLite 또는 duckdb와 같은 *프로세스 내* DBMS는 전적으로 컴퓨터에서 실행됩니다. 사용자가 주 사용자인 대용량 데이터 세트로 작업하는 데 유용합니다.

## 데이터베이스에 연결하기

파이썬에서 데이터베이스에 연결하는 데는 여러 가지 옵션이 있지만 연결하려는 데이터베이스 *종류*에 따라 다릅니다. 그러나 단계(예: 데이터베이스에 연결, 데이터 업로드, SQL 시리즈 실행)는 유사하므로 SQLite 데이터베이스라는 한 가지 예만 선택해도 많은 것을 잃지 않을 것입니다. SQLite는 작고 빠르며 독립적인 SQL 데이터베이스 엔진이며 세계에서 가장 많이 사용되는 데이터베이스 엔진입니다. 컴퓨터와 휴대폰의 많은 데이터는 실제로 SQLite 데이터베이스에 있습니다. (Simon Willison은 사용자가 만든 데이터 일부를 내보내는 데 도움이 되는 [훌륭한 도구](https://dogsheep.github.io/)를 가지고 있습니다!)

그러나 연결하는 SQL 데이터베이스 종류에 관계없이 항상 두 단계가 필요합니다.

-   데이터베이스에 대한 연결을 제공하는 데이터베이스 인터페이스(예: 파이썬 내장 **sqlite** 패키지)를 항상 사용합니다.

-   데이터베이스에서 데이터를 푸시하거나 가져오는 패키지(예: **pandas**)도 사용합니다.

연결의 정확한 세부 정보는 DBMS마다 매우 다르므로 안타깝게도 여기서는 모든 세부 정보를 다룰 수 없습니다. 초기 설정은 종종 약간의 조정(그리고 약간의 조사)이 필요하지만 일반적으로 한 번만 수행하면 됩니다. 여기서는 몇 가지 기본 사항을 다루기 위해 최선을 다할 것입니다.

클라이언트-서버 또는 클라우드 DBMS를 설정하는 것은 이 책에는 번거로운 일이므로 대신 컴퓨터 내 DBMS를 사용합니다.

## 데이터베이스와 직접 작업하기

음악 상점의 아티스트, 노래 및 앨범에 대한 정보와 상점 직원, 고객 및 고객 구매에 대한 정보가 포함된 [Chinook 데이터베이스](https://database.guide/2-sample-databases-sqlite/)라는 작은 SQLite 데이터베이스에 연결해 보겠습니다. 이 정보는 11개의 테이블에 포함되어 있습니다. 아래 그림은 데이터 스키마를 보여줍니다.

![Chinook 데이터베이스의 데이터 스키마](https://github.com/arjunchndr/Analyzing-Chinook-Database-using-SQL-and-Python/raw/master/chinook-schema.svg)

이 책의 github 저장소 [여기](https://github.com/aeturrell/python4DS/tree/main/data)에서 Chinook.sqlite 파일을 다운로드할 수 있습니다. 컴퓨터에서는 아래 연습을 수행하기 위해 코드가 실행되는 위치에 상대적인 'data'라는 하위 디렉터리에 저장해야 합니다.

우선 파이썬 내장 **sqlite3** 엔진을 사용하여 데이터베이스에 연결하고 매우 간단한 SQL 쿼리를 실행하여 'Artists' 테이블에서 처음 10개 항목을 선택합니다.

In [ ]:
import sqlite3

con = sqlite3.connect(Path("data/Chinook.sqlite"))

cursor = con.execute("SELECT * FROM Artist LIMIT 10;")
rows = cursor.fetchall()
rows

여기서 출력은 튜플이라는 파이썬 객체 형식입니다. 이것을 **pandas** 데이터 프레임에 넣고 싶다면 바로 전달하면 됩니다.

In [ ]:
import pandas as pd

pd.DataFrame(rows)

또 다른 유용한 힌트는 열 이름이 확실하지 않은 경우 다음에서 가져올 수 있다는 것입니다.

In [ ]:
[i[0] for i in cursor.description]

### 데이터베이스 만들기

나중에 데이터 일부에 (효율적으로) 액세스하기 위해 SQL 데이터베이스를 만들고 싶을 때가 많습니다. **sqlite** 패키지를 사용하여 직접 테스트 데이터베이스를 만들어 보겠습니다. 이 프로세스에는 `CREATE TABLE` 문, 테이블 이름, 열 이름 및 해당 데이터 유형이 포함됩니다.

In [ ]:
create_query = """CREATE TABLE test (country VARCHAR(20), gdp REAL, health INTEGER);"""
con_new = sqlite3.connect("data/test_database.sqlite")
con_new.execute(create_query)
con_new.commit()

이것이 실행되면 테스트 데이터베이스를 만든 것입니다! (자신의 컴퓨터에서 데이터 디렉터리를 확인하여 작동했는지 확인할 수 있지만 데이터 디렉터리가 아직 없으면 오류가 발생합니다.)

이제 일부 값으로 데이터베이스를 채워 보겠습니다.

In [ ]:
test_data = [("US", 1, 3), ("UK", 0.6, 2), ("France", 0.8, 1)]

con_new.executemany("INSERT INTO test VALUES(?, ?, ?)", test_data)
con_new.commit()

마지막으로 이것이 작동했는지 확인해 보겠습니다.

In [ ]:
con_new.execute("SELECT * FROM test").fetchall()

와, 성공했습니다!

In [ ]:
# 셀 삭제

import os

os.remove("data/test_database.sqlite")

## 기본 SQL 쿼리

SQL 쿼리에 대한 전체 설명은 이 책의 범위를 벗어나지만 기본 사항을 알려 드리겠습니다. SQL의 최상위 구성 요소는 **문**이라고 합니다. 일반적인 문에는 새 테이블을 정의하기 위한 `CREATE`, 데이터를 추가하기 위한 `INSERT`, 데이터를 검색하기 위한 `SELECT`가 있습니다. 데이터 과학자로서 거의 독점적으로 사용하게 될 **쿼리**라고도 하는 `SELECT` 문에 중점을 둘 것입니다.

쿼리는 **절**로 구성됩니다. 중요한 절은 `SELECT`, `FROM`, `WHERE`, `ORDER BY`, `GROUP BY`, `LIMIT`의 6가지입니다. 모든 쿼리에는 `SELECT`[^import-databases-4] 및 `FROM`[^import-databases-5] 절이 있어야 하며 가장 간단한 쿼리는 `SELECT * FROM table`이며, 이는 "table"이라는 지정된 테이블에서 모든 열을 선택합니다. `WHERE` 및 `ORDER BY`는 포함할 행과 정렬 방법을 제어합니다. `GROUP BY`는 쿼리를 요약으로 변환하여 집계를 발생시킵니다. `LIMIT`는 반환되는 행 수를 제한합니다.

중요: SQL에서는 순서가 중요합니다. 항상 `SELECT`, `FROM`, `WHERE`, `GROUP BY`, `ORDER BY` 순서로 절을 작성해야 합니다. 혼란스럽게도 이 순서는 실제로 절이 평가되는 방식과 일치하지 않습니다. 즉, 먼저 `FROM`, 그 다음 `WHERE`, `GROUP BY`, `SELECT`, `ORDER BY` 순서입니다.


[^import-databases-4]: 혼란스럽게도 컨텍스트에 따라 `SELECT`는 문 또는 절일 수 있습니다.
    이러한 혼동을 피하기 위해 일반적으로 `SELECT` 문 대신 쿼리를 사용합니다.

[^import-databases-5]: 기술적으로는 `SELECT 1+1`과 같은 쿼리를 작성하여 기본 계산을 수행할 수 있으므로 `SELECT`만 필요합니다. 그러나 데이터로 작업하려면(항상 그렇듯이!) `FROM` 절도 필요합니다.

이들 중 몇 가지를 실제로 살펴보겠습니다. 몇 가지 항목을 가져오려면 다음과 같습니다.

```sql
SELECT * FROM Artist LIMIT 10;
```

열을 지정하려면 다음과 같습니다.

```sql
SELECT name FROM Artist LIMIT 10;
```

순서를 바꾸려면 "ORDER BY"를 사용한 다음 "DESC"를 사용합니다. 예를 들어 이 쿼리는 가장 긴 트랙 10개를 제공합니다.

```sql
SELECT name, milliseconds FROM track ORDER BY milliseconds DESC LIMIT 10;
```

또 다른 주요 기능은 필터링 기능입니다. 예를 들어 길이가 3분 이상인 처음 10개 트랙을 요청할 수 있습니다. 실제로 실행해 보겠습니다.

In [ ]:
sql_query = "SELECT name, milliseconds FROM track WHERE milliseconds > 1e3*3*60 ORDER BY milliseconds ASC LIMIT 10;"
cursor = con.execute(sql_query)
rows = cursor.fetchall()
rows

groupby를 시도해 보겠습니다. 이것은 앨범별로 그룹화하여 (처음 5개) 앨범의 평균 트랙 길이를 분 단위로 찾습니다.

In [ ]:
sql_groupby = "SELECT albumid, AVG(milliseconds)/1e3/60 FROM track GROUP BY albumid ORDER BY AVG(milliseconds) ASC LIMIT 5;"
cursor = con.execute(sql_groupby)
rows = cursor.fetchall()
rows

### 조인

**pandas**의 조인에 익숙하다면 SQL 조인은 매우 유사합니다. 'album' 테이블과 'track' 테이블을 조인하여 위 쿼리의 앨범 *이름*을 찾을 수 있는지 살펴보겠습니다.

두 개 이상의 테이블에 *동일한* 열 이름이 있는 즉시 해당 열 이름을 사용할 때 참조하는 테이블을 지정해야 합니다. 조인에는 다양한 옵션(예: `INNER`, `LEFT`)이 있으며 [여기](https://en.wikipedia.org/wiki/Join_(SQL))에서 자세히 알아볼 수 있습니다.


In [ ]:
sql_join = "SELECT track.albumid, AVG(milliseconds)/1e3/60, album.title FROM track INNER JOIN album ON (track.albumid = album.albumid) GROUP BY album.albumid ORDER BY AVG(milliseconds) ASC LIMIT 5;"
cursor = con.execute(sql_join)
rows = cursor.fetchall()
rows

### 함수

이미 함수 하나를 몰래 사용했습니다. `AVG`. 열에 적용할 수 있는 다른 함수에는 `SUM`, `MIN`, `MAX`, `FIRST`, `LAST` 및 `COUNT`(데이터 과학자의 친구)가 있습니다. SQL 함수에 대한 자세한 내용은 [여기](https://www.tutorialspoint.com/sql/sql-useful-functions.htm)에서 확인할 수 있습니다.

## **Datasette**으로 SQL 데이터베이스 탐색하기

SQL은 처음에는 상당히 위협적일 수 있지만 [**Datasette**](https://datasette.io/)이라는 더 잘 알 수 있는 훌륭한 도구가 있습니다. Datasette은 데이터베이스를 탐색하고(만들고 공유하는) 도구입니다. 저자는 "Datasette은 데이터 저널리스트, 박물관 큐레이터, 기록 보관 담당자, 지방 정부, 과학자, 연구원 및 세상과 공유하고 싶은 데이터를 가진 모든 사람을 대상으로 합니다. 구조화된 데이터 작업을 최대한 생산적으로 만들기 위해 전념하는 40개의 도구와 99개의 플러그인으로 구성된 더 넓은 생태계의 일부입니다."라고 말합니다.

이러한 맥락에서 **Datasette**은 두 가지 방식으로 유용합니다.

- 포인팅과 클릭을 통해 데이터베이스를 대화형으로 탐색합니다.
- SQL `SELECT` 문이 필터링 작업으로 어떻게 구성되는지 또는 그 반대로 어떻게 구성되는지 확인합니다.

기본적으로 **Datasette**이 실행 중이면 데이터 세트의 모든 테이블을 보여주는 방문 웹 페이지가 표시됩니다(예, 자신의 컴퓨터에서 실행 중인 경우에도). 테이블을 클릭하여 탐색하고, 순서를 변경하고, 필터링하는 등의 작업을 수행할 수 있습니다. 또한 만든 모든 컷에 대한 동등한 SQL 쿼리를 보거나 상자에 SQL 쿼리를 실행하여 원하는 모든 컷을 검색할 수 있습니다. 단, **Datasette**은 `SELECT` 문만 지원합니다.

유용한 추가 기능은 원하는 대로 필터링한 후 **Datasette**에서 실행되는 웹 페이지에서 SQL 쿼리 결과를 CSV 또는 JSON 파일로 다운로드할 수 있다는 것입니다.

**Datasette**을 사용하여 데이터를 탐색하는 몇 가지 다른 방법이 있습니다.

- 자신의 컴퓨터에서 실행할 수 있습니다.
- 온라인 버전(이미 클라우드에 호스팅됨)을 사용해 볼 수 있습니다. 예를 들어 발전소의 [이 데이터베이스](https://global-power-plants.datasettes.com/global-power-plants/global-power-plants)가 있습니다.
- 온라인 코딩 서비스 glitch를 사용하여 실행할 수 있습니다. [여기](https://glitch.com/~datasette-csvs)에서 예를 참조하십시오.

**Datasette**은 명령줄에서 `pip install datasette`를 실행하여 설치할 수 있는 파이썬 패키지로 제공됩니다. 파이썬 환경에 설치한 후 다음을 실행합니다.

```bash
datasette path/to/database.db -o
```

그러면 기본 브라우저가 데이터베이스의 테이블을 보여주는 페이지에서 즉시 열립니다(페이지 주소는 `http://localhost:8001/`로 시작해야 함).

마지막으로 **Datasette**을 사용하는 SQL 교육이 [여기](https://datasette.io/tutorials/learn-sql)에 있습니다.

````{admonition} 연습
자신의 컴퓨터에서 Chinook 데이터베이스를 사용하여 이전에 조인한 SQL 쿼리를 실행합니다. 즉,

```sql
SELECT
  albumid,
  AVG(milliseconds) / 1e3 / 60
FROM
  track
GROUP BY
  albumid
ORDER BY
  AVG(milliseconds) ASC
LIMIT
  5
```

````

## **pandas**로 SQL 사용하기

**pandas**는 SQL 작업에 적합합니다. 방금 만든 쿼리를 `read_sql()` 함수를 사용하여 바로 푸시할 수 있습니다. 단, 데이터베이스에 만든 연결도 전달해야 한다는 점에 유의하십시오.

In [ ]:
pd.read_sql(sql_join, con)

이것의 좋은 점 중 하나는 SQL의 열 이름이 데이터 프레임의 열 이름으로 바로 전달된다는 것입니다.

이제 Visual Studio Code에서 파이썬을 작성할 때(적어도 파이썬 확장이 설치된 경우) 고품질 구문 및 자동 완성 지원을 많이 받습니다. 파이썬 언어 확장을 사용하면 처리하는 변수 유형에 대해 매우 주의를 기울일 수도 있습니다. SQL에서도 이 모든 것을 사용할 수 있다면 좋지 않을까요(파이썬을 통해 액세스하는 경우에도)? 다음에 살펴볼 두 패키지가 이를 제공합니다. 두 패키지 모두 파이썬에서 SQL 데이터베이스로 작업하는 것을 훨씬 쉽고 생산적으로 만듭니다.

## **ibis**로 SQL 사용하기

SQL 쿼리를 텍스트로 작성해야 하는 것은 그다지 만족스럽지 않습니다. **pandas** 명령에서 직접 명령을 만들 수 있다면 어떨까요? 그렇게 할 수는 없지만 [**ibis**](https://ibis-project.org/)라는 패키지를 사용하면 거의 비슷하게 할 수 있습니다. **ibis**는 데이터베이스에서 읽고 **pandas** 데이터 프레임처럼 쿼리하려는 경우 특히 유용합니다.

**Ibis**는 로컬 데이터베이스(예: SQLite 데이터베이스), 서버 기반 데이터베이스(예: Postgres) 또는 클라우드 기반 데이터베이스(예: Google의 BigQuery)에 연결할 수 있습니다. 연결을 만드는 구문은 예를 들어 `ibis.bigquery.connect`입니다.

로컬에서 호스팅되는 데이터베이스인 Chinook 데이터베이스에서 이미 본 명령을 재현하여 **ibis**를 실제로 살펴보겠습니다. 먼저 가져와서 데이터베이스에 연결합니다.

In [ ]:
import ibis

ibis.options.interactive = True
connection = ibis.sqlite.connect("data/Chinook.sqlite")
track = connection.table("track")
track.head()

자, 이제 다음 쿼리를 재현해 보겠습니다. "SELECT albumid, AVG(milliseconds)/1e3/60 FROM track GROUP BY albumid ORDER BY AVG(milliseconds) ASC LIMIT 5;". groupby, mutate(**pandas**의 할당 문과 유사하다고 생각할 수 있음), 정렬, 그런 다음 `limit()`를 사용하여 처음 5개 항목만 표시합니다.

In [ ]:
track.group_by("AlbumId").mutate(
    mean_mins_track=track.Milliseconds.mean() / 1e3 / 60
).order_by("mean_mins_track").limit(5)

In [ ]:
track.group_by("AlbumId").mutate(mean_mins_track=track.Milliseconds.mean() / 1e3 / 60)

조인은 어떻습니까? 물론 이것도 할 수 있습니다. 예를 들어 공유 변수인 "GenreId"를 기준으로 장르 테이블과 트랙 테이블을 조인해 보겠습니다.

In [ ]:
genre = connection.table("genre")
genre_and_track = track.inner_join(
    genre, predicates=track["GenreId"] == genre["GenreId"]
)

genre_and_track

**Ibis**는 지연 평가, 지리 공간 작업, 타이핑 및 물론 예상되는 모든 일반적인 SQL 명령을 포함하여 여기에 표시된 것보다 훨씬 많은 작업을 수행합니다. 자세한 내용은 [설명서](https://ibis-project.org/)를 확인하십시오.

## **SQLModel**로 SQL 사용하기

**Ibis**만이 파이썬에서 SQL로 작업하는 현명한 옵션은 아닙니다. [**SQLModel**](https://sqlmodel.tiangolo.com/)도 또 다른 옵션입니다. 프로덕션 등급 SQL 데이터베이스를 만들거나 매우 풍부한 지원을 통해 **ibis**와 약간 다른 구문 스타일을 사용하여 쿼리하려는 경우 **sqlmodel**을 사용합니다.

따라서 **sqlmodel**의 몇 가지 주요 강점에는 환상적인 자동 완성 지원과 데이터 유형에 대한 매우 엄격함(특히 데이터베이스를 *만드는* 경우 장기적으로 시간을 절약해 줌)이 포함됩니다.

먼저 명령줄에서 `pip install sqlmodel`을 실행하여 패키지가 설치되어 있는지 확인합니다.

### **SQLModel**로 SQL 데이터베이스 만들기

"hero"라는 SQL 테이블이 있고 이 데이터가 포함되기를 원한다고 상상해 보십시오.

| id | name | secret_name | age |
|---|---|---|---|
| 1 | Deadpond | Dive Wilson | null |
| 2 | Spider-Boy | Pedro Parqueador | null |
| 3 | Rusty-Man | Tommy Sharp | 48 |
| 4 | Ms Amazing | Barjabeen Bhabra | 17 |

이제 SQL 테이블을 만들려면 *클래스*라는 구조를 작성합니다. 클래스는 함수와 약간 비슷하지만 이 클래스의 본문에는 필드 이름과 해당 데이터 유형에 대한 파이썬 코드가 나열됩니다.

In [ ]:
# 셀 삭제

import warnings

warnings.filterwarnings(
    "ignore", ".*Class SelectOfScalar will not make use of SQL compilation caching.*"
)

In [ ]:
from typing import Optional

from sqlmodel import Field, Session, SQLModel, create_engine


class Hero(SQLModel, table=True):
    id: Optional[int] = Field(default=None, primary_key=True)
    name: str
    secret_name: str
    age: Optional[int] = None

그런 다음 데이터 행을 만듭니다(프로그래밍 방식으로 수행할 수 있지만 여기서는 직접 작성합니다). 아래 필드에 자동 완성이 있다는 점에 유의하십시오!

In [ ]:
hero_1 = Hero(name="Deadpond", secret_name="Dive Wilson")
hero_2 = Hero(name="Spider-Boy", secret_name="Pedro Parqueador")
hero_3 = Hero(name="Rusty-Man", secret_name="Tommy Sharp", age=48)
hero_4 = Hero(name="Ms Amazing", secret_name="Barjabeen Bhabra", age=17)

이제 실제 데이터베이스를 만들고 연결해야 합니다. 구문은 데이터베이스 유형의 이름(여기서는 sqlite)이고 그 뒤에 `:///`와 데이터베이스를 저장하려는 상대 경로가 옵니다.

In [ ]:
engine = create_engine("sqlite:///data/hero.db")

SQLModel.metadata.create_all(engine)

with Session(engine) as session:
    session.add(hero_1)
    session.add(hero_2)
    session.add(hero_3)
    session.add(hero_4)
    session.commit()

이제 직접적인 접근 방식(즉, **sqlite3** 패키지 사용)을 사용하여 만든 데이터베이스에서 판독값을 가져올 수 있습니다.

In [ ]:
con = sqlite3.connect(Path("data/hero.db"))

con.execute("SELECT * FROM hero;").fetchall()

### **SQLModel**을 사용하여 SQL에서 읽기

**SQLModel**은 일반적인 모든 SQL 문을 함수로 변환합니다. "SELECT"도 다르지 않습니다. 따라서 **SQLModel**에서 직접 SQL 데이터베이스를 읽는 예를 살펴보겠습니다.

In [ ]:
from sqlmodel import select

with Session(engine) as session:
    statement = select(Hero)  # SELECT * from hero와 동일
    results = session.exec(statement)  # 명령 실행
    for hero in results:
        print(hero)

`results.all()`을 통해 모든 것을 다시 가져올 수도 있습니다. 결과 객체는 "hero" 객체 목록이며 해당 속성은 `heroes[0].name` 등을 통해 액세스할 수 있습니다.

In [ ]:
with Session(engine) as session:
    statement = select(Hero)
    results = session.exec(statement)
    heroes = results.all()

print(*heroes, sep="\n")

**SQLModel**에 대한 자세한 내용은 [설명서](https://sqlmodel.tiangolo.com/)에서 확인할 수 있지만 예상되는 대부분의 SQL 쿼리 명령을 사용할 수 있습니다. 이전 예제를 실행해 보겠습니다.

In [ ]:
with Session(engine) as session:
    statement = select(Hero).where(Hero.age < 100).limit(2).order_by(Hero.age)
    results = session.exec(statement)
    heroes = results.all()

print(*heroes, sep="\n")

이것은 **SQLModel**이 할 수 있는 일의 일부만 다루었습니다. 조인, 데이터 업데이트, 행 삭제 등도 모두 가능합니다.

In [ ]:
# 셀 삭제

import os

os.remove("data/hero.db")

## SQL에 대해 더 자세히 알아볼 수 있는 곳

이 장을 마치고 SQL에 대해 더 자세히 알고 싶다면 다음과 같은 추가 자료가 있습니다.

-   Renée M. P. Teate의 [*데이터 과학자를 위한 SQL*](https://sqlfordatascientists.com)은 데이터 과학자의 요구 사항에 맞게 특별히 설계된 SQL 소개이며 실제 조직에서 접하게 될 가능성이 높은 고도로 상호 연결된 데이터의 예를 포함합니다.
-   Anthony DeBarros의 [*실용 SQL*](https://www.practicalsql.com)은 데이터 저널리스트(매력적인 이야기를 전달하는 데 특화된 데이터 과학자)의 관점에서 작성되었으며 데이터를 데이터베이스에 넣고 자체 DBMS를 실행하는 방법에 대해 자세히 설명합니다.
-   모든 기본 사항이 한 곳에 작성된 많은 [*sql 치트 시트*](https://learnsql.com/blog/sql-basics-cheat-sheet/sql-basics-cheat-sheet-a3.pdf) 중 하나를 확인하십시오.
-   [SQLime](https://sqlime.org/)에서 브라우저에서 sql을 사용해 보십시오.
-   [살인 미스터리](https://mystery.knightlab.com/)를 해결하여 SQL을 배우십시오.